In [1]:
import cvxpy as cp

##Aiden Pulsifer, Jake Denofsky, Leon Chang

##x_(i,j) ordered by descending order of city j's attendance value, then a secondary order by city i's attendance value,
##for all edges between cities under 500 miles.
##example: x[0] represents if Taylor Swift's tour goes from Charlotte (i) to Columbus (j). 
##x[265] represents if Taylor Swift's tour goes from Boise (i) to Portland (j)
##133 edges so 266 combos of i,j
x = cp.Variable(266, boolean = True)

##y ordered in descending order of attendance
##example: y[0] represents if tour started in Columbus, OH, y[41] represents if tour started in Portland, OR
y = cp.Variable(42, boolean = True)

##boolean variables to enforce if statement that forces the second show to be traveled to from city i such that y_i = 1
##if statement: if Taylor Swift starts her tour at city i, she should travel away from city i to get to the city of her second show.
h = cp.Variable(42, boolean = True)
b = cp.Variable(42, boolean = True)
z = cp.Variable(42, boolean = True)

##array to hold capacities of stadiums, ordered by descending attendance
stadiumCapacities = [102780, 82500, 80000, 77500, 76414, 76125,74867,73208,72220,71835,71000,70240,69218,69143,68740,68500,68400,67814, 67594, 67431, 67277, 66655, 65878, 65632, 65515, 65000, 64767, 64000, 63400, 63000, 62000, 61500, 60219, 51444, 50000, 49262, 42445, 41800, 41503, 36387, 35000, 32000]

##initialize obj_func to use in for loop
obj_func = 0
for i in range(42):
    obj_func += stadiumCapacities[i]*y[i]

###Add x variable summation into obj_func
##Make array with the amount of edges to each city i.e. [12,9,2,6, etc.]: Columbus has 12 edges to it, East Rutherford has 9, Arlington has 2, etc.
##This represents the values of variable a in the formulation
edgeArray = [12,9,2,6,4,2,8,2,3,5,8,6,4,8,2,4,14,5,10,13,7,2,6,13,12,8,3,2,4,10,12,9,4,3,3,9,6,8,4,3,9,2]
counter = 0
for i in range(42):
    for j in range(edgeArray[i]):
        obj_func += stadiumCapacities[i]*x[counter]
        counter+=1
    
##initialize constraints object
constraints = []

#Constraint 1: sum(y_i) = 1 -> Taylor Swift can only start from 1 city and sum(z_i) = 1 -> Taylor Swift can only end at 1 city
constraints.append(sum(y)==1)
constraints.append(sum(z)==1)

#Constraint 2: sum(x_(i,j)) = 11 -> Taylor Swift should play 12 shows on her tour
constraints.append(sum(x)==11)

#Constraint 3: Taylor Swift can play at each city only once at most 
counter = 0
for i in range(42):
    ##Initialize equation variable with y_i variable
    equation = y[i]
    for j in range(edgeArray[i]):
        ##Add sum of x variables for that city destination to equation
        equation+=x[counter]
        counter+=1
    ##Add constraint 
    constraints.append(equation<=1)

##2d array that connects the cities with the edges they go to, first array of the array is the indices of x in which Taylor Swift goes from Columbus to another city
##Index = “edge” column of google sheet - 2
outgoingEdgeArray = [[35, 71, 85, 104, 114, 127, 142, 155, 167, 184, 194, 206], [86, 105, 115, 136, 143, 195, 225, 240, 255], [45, 178], [48, 61, 81, 180, 234, 248], [128, 134, 185, 222], [219, 223], [0, 53, 72, 87, 99, 144, 156, 196], [46, 54], [21, 43, 179], [23, 62, 181, 220, 235], [36, 44, 67, 73, 100, 145, 157, 215], [24, 49, 82, 182, 236, 249], [55, 101, 175, 216], [1, 37, 56, 129, 146, 158, 186, 207], [252, 264], [25, 63, 237, 250], [2, 12, 38, 106, 116, 147, 159, 168, 187, 197, 208, 226, 241, 256], [39, 57, 68, 176, 217], [3, 13, 88, 117, 137, 148, 198, 227, 242, 257], [4, 14, 89, 107, 149, 160, 169, 188, 199, 209, 228, 243, 258], [5, 29, 74, 161, 189, 210, 224], [30, 211], [15, 108, 200, 229, 244, 259], [6, 16, 40, 58, 75, 90, 109, 118, 162, 170, 190, 201, 260], [7, 41, 59, 76, 91, 119, 130, 150, 171, 191, 202, 212], [8, 92, 120, 151, 163, 192, 213, 230], [69, 102, 218], [22, 47], [26, 50, 64, 238], [9, 31, 77, 94, 122, 131, 153, 165, 172, 214], [10, 17, 42, 93, 110, 121, 138, 152, 164, 231, 245, 261], [11, 78, 95, 123, 132, 135, 166, 173, 193], [60, 70, 103, 177], [33, 51, 253], [32, 34, 133], [18, 96, 111, 124, 139, 174, 203, 246, 262], [27, 52, 65, 83, 183, 251], [19, 97, 112, 125, 140, 204, 232, 263], [28, 66, 84, 239], [79, 221, 265], [20, 98, 113, 126, 141, 154, 205, 233, 247], [80, 254]] 
    

#Constraints 4 and 5: if Taylor Swift starts her tour at city i, she should travel away from city i to get to the city of her second show.
#Constraint 4:
##Loop through each array of outgoingEdgeArray
for i in range(42):
    ##Initialize equation variable with 0
    equation = 0
    for j in range(len(outgoingEdgeArray[i])):
        equation += x[outgoingEdgeArray[i][j]]
    equation += -1*h[i]
    constraints.append(equation>=0)
    
    
#Constraint 5: second constraint of if statement construction
for i in range(42):
    constraints.append(y[i]-h[i]==0)

##Loop through each array of outgoingEdgeArray
counter = 0
for i in range(42):
    ##Initialize equation variable with 0
    equation = 0
    for j in range(edgeArray[i]):
        ##Add sum of x variables for that city destination to equation
        equation+=x[counter]
        counter+=1
    equation += -1*b[i]
    constraints.append(equation>=0)
    
    
for i in range(42):
    constraints.append(z[i]-b[i]==0)

##Conservation constraint: edges(in)=edges(out) for each city

counter = 0;
for i in range(42):
    inVariable = 0
    outVariable = 0
    for j in range(edgeArray[i]):
        inVariable += x[counter]
        counter+=1
        sumIn = sum(inVariable)
    for k in range(len(outgoingEdgeArray[i])):
        outVariable += x[outgoingEdgeArray[i][k]]
        sumOut = sum(outVariable)

    
    constraints.append(y[i]+inVariable-outVariable-z[i]==0)


##MUST LIMIT CYCLES TO KEEP HER FROM TELEPORTING
##limit all 2 city cycles
counter = 0
for i in range(42):
    for j in range(edgeArray[i]):
        constraints.append(x[counter]+x[outgoingEdgeArray[i][j]]<=1)
        counter+=1

##Lazy constraints for cycles of length 3+
constraints.append(x[3]+x[105]+x[12]+x[35]+x[87]<=4)

constraints.append(x[24]+x[48]+x[62]<=2)

constraints.append(x[12]+x[105]+x[88]<=2)
constraints.append(x[13]+x[106]+x[86]<=2)

constraints.append(x[23]+x[61]+x[49]<=2)

constraints.append(x[25]+x[48]+x[62]+x[82]<=3)
constraints.append(x[49]+x[23]+x[81]+x[63]<=3)

constraints.append(x[81]+x[63]+x[24]<=2)
constraints.append(x[82]+x[61]+x[25]<=2)

##cycle of four vertices between glendale, la, paradise, and inglewood
constraints.append(x[182] + x[26] + x[48] + x[62] <=3)

constraints.append(x[182] + x[50] + x[23] + x[61] <=3)

constraints.append(x[181] + x[49] + x[26] + x[61] <=3)

constraints.append(x[180] + x[49] + x[23] + x[64] <=3)

constraints.append(x[181] + x[48] + x[24] + x[64] <=3)

constraints.append(x[180] + x[50] + x[24] + x[62] <=3)
                   
##cycle of three between la, glendale, paradise
constraints.append(x[180] + x[50] + x[23] <=2)
constraints.append(x[181] + x[48] + x[26] <=2)

##cycle of three between la, glendale, inglewood
constraints.append(x[26] + x[61] + x[182] <=2)
constraints.append(x[24] + x[64] + x[180] <=2)

##cycle of three between paradise, glendale, inglewood
constraints.append(x[181] + x[49] + x[64] <=2)
constraints.append(x[182] + x[50] + x[62] <=2)


problem = cp.Problem(cp.Maximize(obj_func),constraints)
problem.solve(solver=cp.GUROBI, verbose=True)

#Print out results
print("obj_func =")
print(obj_func.value)
print("x =")
print(x.value)

##Print out starting city’s index
print("Starting index:")
for i in range(42):
    if(y.value[i]==1):
        print(i+1)

print("Edge indices in numerical order:")
##Print out indices of edges
for i in range(266):
    if(x.value[i]==1):
        print(i+2)

print("Ending index:")
##Print out ending city’s index
for i in range(42):
    if(z.value[i]==1):
        print(i+1)


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 12:54:54 PM: Your problem has 434 variables, 584 constraints, and 0 parameters.
(CVXPY) Nov 25 12:54:54 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 12:54:54 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 12:54:54 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 12:54:54 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 12:54:54 PM: Compiling problem (target solver=GUROBI).